In [12]:
import get_api_data,git_local
import pandas as pd
from flatten_dict import flatten
from datetime import datetime
from dateutil import relativedelta
import random as rd

In [20]:
# Mock file creation
access_token = 'ef66b5bf5fd6c14d63586bf00caa9e72c78cc675'
repo_owner = 'rspec'
git_url = 'git://github.com/rspec/rspec-rails.git'
api_base_url = 'https://api.github.com'
repo_name = 'rspec-rails'
df = pd.read_excel('uc2.xlsx')

In [21]:
df.drop(['DestinationFilePath','NormalizedChange'],inplace = True, axis = 1)

In [22]:
#df.drop_duplicates(subset = ['Source_file'],inplace = True)
df = df.groupby('SourceFilePath',as_index=False).sum()

In [23]:
df.drop_duplicates(inplace=True)
df['BuggyCommits'] = [0]*df.shape[0]
df['BuggyCommitsPercentage'] = [0]*df.shape[0]
df['RepoName'] = [0]*df.shape[0]
df['RepoUrl'] = [0]*df.shape[0]

In [24]:
buggy_commit = []
for i in range(df.shape[0]):
    df.iloc[i,2] = int(round(df.iloc[i,1]*rd.random()))
    df.iloc[i,3] = df.iloc[i,2]/df.iloc[i,1]
    df.iloc[i,4] = repo_name
    df.iloc[i,5] = git_url

In [27]:
df.to_csv('uc3.csv')

In [26]:
df

,SourceFilePath,NoOfTimeChanged,BuggyCommits,BuggyCommitsPercentage,RepoName,RepoUrl
0,.document,15,2,0.133333,rspec-rails,git://github.com/rspec/rspec-rails.git
1,.gitignore,146,3,0.020548,rspec-rails,git://github.com/rspec/rspec-rails.git
2,.rspec,9,1,0.111111,rspec-rails,git://github.com/rspec/rspec-rails.git
3,.rubocop.yml,88,35,0.397727,rspec-rails,git://github.com/rspec/rspec-rails.git
4,.rubocop_rspec_base.yml,396,239,0.603535,rspec-rails,git://github.com/rspec/rspec-rails.git
5,.rubocop_todo.yml,90,9,0.100000,rspec-rails,git://github.com/rspec/rspec-rails.git
6,.travis.yml,269,5,0.018587,rspec-rails,git://github.com/rspec/rspec-rails.git
7,.yardopts,77,52,0.675325,rspec-rails,git://github.com/rspec/rspec-rails.git
8,BUILD_DETAIL.md,12,8,0.666667,rspec-rails,git://github.com/rspec/rspec-rails.git
9,CAPYBARA.md,2,0,0.000000,rspec-rails,git://github.com/rspec/rspec-rails.git


In [ ]:
def UC3(committed_files_df):
    temp_df = committed_files_df
    temp_df = temp_df.drop(labels = ['file_id','modification_type'],axis = 1)
    uc2 = {}
    total_file_changed = {}
    total_bugs = {}
    for commit in temp_df.commit_id.unique():
        files = temp_df[temp_df['commit_id'] == commit].file_name.values.tolist()
        buggy = temp_df[temp_df['commit_id'] == commit].bug_fixing.values.tolist()[0]
        for file_s in files:
            _file = os.getcwd() + '/temp_repo/' + repo_name + '/' + file_s
            exists = os.path.isfile(_file)
            if exists:
                if file_s not in total_file_changed:
                    total_file_changed[file_s] = 1
                    total_bugs[file_s] = 0
                    if buggy:
                        total_bugs[file_s] += 1
                else:
                    total_file_changed[file_s] += 1
                    if buggy:
                        total_bugs[file_s] += 1
            else:
                continue
                
    uc2_matrix = []
    for file_s in total_file_changed:
            uc2_matrix.append([file_s,total_file_changed[file_s],total_bugs[file_s],
                               total_bugs[file_s]/total_file_changed[file_s],repo_name,git_url])
    uc2_matrix_df = pd.DataFrame(uc2_matrix, columns = ['SourceFilePath','NoOfTimeChanged','BuggyCommits',
                                                        'BuggyCommitsPercentage','RepoName','RepoUrl'])
    return uc2_matrix_df